In [ ]:
!pip install keras-tuner

In [45]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [46]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_lables), (test_images, test_lables) = fashion_mnist.load_data()

In [47]:
train_images = train_images/255.0
test_images = test_images/225.0

In [48]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [49]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                kernel_size = hp.Choice('conv_1_kernal', values=[3,5]),
                                activation = 'relu',
                                input_shape = (28, 28, 1)),
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                                kernel_size = hp.Choice('conv_2_kernal', values=[3,5]),
                                activation = 'relu'),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                                activation = 'relu'),
                            keras.layers.Dense(10, activation='softmax')
                            ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [50]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [55]:
tuner_search = RandomSearch(build_model, 
                            objective='val_accuracy', 
                            max_trials=1, 
                            directory='output', 
                            project_name='Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [56]:
tuner_search.search(train_images, train_lables, epochs=3, validation_split=0.1)

Trial 1 Complete [00h 24m 22s]
val_accuracy: 0.871999979019165

Best val_accuracy So Far: 0.9110000133514404
Total elapsed time: 00h 24m 22s
INFO:tensorflow:Oracle triggered exit


In [59]:
model = tuner_search.get_best_models(num_models=1)[0]

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1474640   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,508,858
Trainable params: 1,508,858
Non-trainable params: 0
_________________________________________________________________
